## Applying the bias scan tool on a loan approval classifier
In this notebook, the bias scan tool is applied on a XGBoost loan approval classifier. The bias scan tool is based on an implementation of the k-means Hierarchical Bias Aware Clustering (HBAC) method\*. The python script `./helper_functions.py` contains functions that execute the bias scan. A conceptual description how the bias scan works, including the rationale why k-means is chosen as a clustering algorithm and paramater choices, can be found in the [bias scan report](https://github.com/NGO-Algorithm-Audit/Bias_scan/blob/master/Bias_scan_tool_report.pdf) (technical documentation).

An XGBoost classifier is trained to make predictions on the German Credit\*\* data set. Details on pre-processing steps performed on this dataset and the trained XGBoost classifier are provided in the `../case_studies/Loan_approval_classifier/GermanCredit_classifier.ipynb` notebook.

\* Misztal-Radecka, Indurkya, *Information Processing and Management*. Bias-Aware Hierarchical Clustering for detecting the discriminated groups of users in recommendation systems (2021).

\*\* the original dataset, in the form provided by Prof. Hofmann, contains categorical/symbolic attributes which is contained in the file "german.data" [[link to dataset]](https://archive.ics.uci.edu/ml/datasets/Statlog+%28German+Credit+Data%29)

### Overview of the notebook
1. Load data and pre-processing
2. Bias scan using k-means clustering
3. Clustering results
4. Statistical testing of inter-cluster difference 

### Load libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Markdown, display

# helper functions
from helper_functions import *

# sklearn
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# welch's t-test
import scipy.stats as stats

### 1. Load data and pre-processing
#### Load data

In [2]:
path = '../classifiers/Loan_approval_classifier/pred_XGBoost.csv'

# read data
entire_dataset = pd.read_csv(path)
entire_dataset.head()

,month,credit_amount,investment_as_income_percentage,residence_since,age,number_of_credits,people_liable_for,sex,status=A11,status=A12,...,skill_level=A172,skill_level=A173,skill_level=A174,telephone=A191,telephone=A192,foreign_worker=A201,foreign_worker=A202,predicted_class,true_class,errors
0,21.0,2993.0,3.0,2.0,28.0,2.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0,0.0,0.0
1,30.0,3656.0,4.0,4.0,49.0,2.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0,0.0,0.0
2,12.0,1255.0,4.0,4.0,61.0,2.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0,0.0,0.0
3,8.0,1414.0,4.0,2.0,33.0,1.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0,0.0,0.0
4,12.0,691.0,4.0,3.0,35.0,2.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0,1.0,1.0


#### Data cleaning

In [3]:
features = entire_dataset.drop(['predicted_class', 'true_class', 'errors'], axis=1)
features.head()

,month,credit_amount,investment_as_income_percentage,residence_since,age,number_of_credits,people_liable_for,sex,status=A11,status=A12,...,housing=A152,housing=A153,skill_level=A171,skill_level=A172,skill_level=A173,skill_level=A174,telephone=A191,telephone=A192,foreign_worker=A201,foreign_worker=A202
0,21.0,2993.0,3.0,2.0,28.0,2.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
1,30.0,3656.0,4.0,4.0,49.0,2.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
2,12.0,1255.0,4.0,4.0,61.0,2.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
3,8.0,1414.0,4.0,2.0,33.0,1.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,12.0,691.0,4.0,3.0,35.0,2.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


#### Scaling data

In [4]:
full_data = init_GermanCredit_dataset(entire_dataset,features)
full_data.head()

,month,credit_amount,investment_as_income_percentage,residence_since,age,number_of_credits,people_liable_for,sex,status=A11,status=A12,...,skill_level=A174,telephone=A191,telephone=A192,foreign_worker=A201,foreign_worker=A202,predicted_class,true_class,errors,clusters,new_clusters
0,0.060492,0.002162,-0.049437,-0.764268,-0.665129,0.943341,-0.447214,-0.755434,-0.623610,-0.592749,...,-0.45257,0.839372,-0.839372,0.195096,-0.195096,0,0.0,0.0,0,-1
1,0.823703,0.254090,0.877513,1.084768,1.180823,0.943341,-0.447214,-0.755434,-0.623610,-0.592749,...,-0.45257,0.839372,-0.839372,0.195096,-0.195096,0,0.0,0.0,0,-1
2,-0.702720,-0.658246,0.877513,1.084768,2.235653,0.943341,-0.447214,-0.755434,-0.623610,-0.592749,...,-0.45257,0.839372,-0.839372,0.195096,-0.195096,0,0.0,0.0,0,-1
3,-1.041925,-0.597829,0.877513,-0.764268,-0.225616,-0.711644,-0.447214,-0.755434,-0.623610,1.687055,...,-0.45257,0.839372,-0.839372,-5.125693,5.125693,0,0.0,0.0,0,-1
4,-0.702720,-0.872556,0.877513,0.160250,-0.049811,0.943341,-0.447214,-0.755434,1.603567,-0.592749,...,-0.45257,0.839372,-0.839372,0.195096,-0.195096,0,1.0,1.0,0,-1


### 2. Bias scan using k-means clustering
Clustering algorithms parameters:

In [5]:
clustering_paramaters = {
    "n_clusters": 3,
    "init": "k-means++",
    "n_init": 20,
    "max_iter": 300,
    "random_state": 10,
}

Specify:
- Minimal splittable cluster size
- Minimal acceptable cluster size

In [6]:
# minimal splittable cluster size
split_cluster_size = round(0.05 * len(full_data))
print("minimal splittable cluster size: ", split_cluster_size)

# minimal acceptable cluster size
acc_cluster_size = round(0.03 * len(full_data))
print("minimal splitacceptabletable cluster size: ", acc_cluster_size)

minimal splittable cluster size:  15
minimal splitacceptabletable cluster size:  9


Performing bias scan using helper functions.

In [7]:
metric = 'accuracy'
iterations_max = 20
x = 0 # initial cluster number
initial_bias = 0
variance_list = []
average_accuracy = bias(full_data, metric)
minimal_splittable_cluster_size = 18
minimal_acceptable_cluster_size = 10
print("average_accuracy is: ", average_accuracy) 

for i in range(1, iterations_max):
    if i != 1:
        variance_list.append(calculate_variance(full_data,metric)) 
    full_data['new_clusters'] = -1
    candidate_cluster = full_data.loc[full_data['clusters'] == x] 

    if len(candidate_cluster) < minimal_splittable_cluster_size:
        x = get_random_cluster(full_data['clusters'])
        continue
    
    # Apply Kmeans 
    kmeans_algo = KMeans(**clustering_paramaters).fit(candidate_cluster.drop(['clusters', 'new_clusters', 'predicted_class', 'true_class', 'errors'], axis=1))
    
    candidate_cluster['new_clusters'] = pd.DataFrame(kmeans_algo.predict(candidate_cluster.drop(['clusters', 'new_clusters', 'predicted_class', 'true_class', 'errors'], axis=1)),index=candidate_cluster.index) 
    full_data['new_clusters'] = candidate_cluster['new_clusters'].combine_first(full_data['new_clusters'])

    max_abs_bias = get_max_bias(full_data,metric) # was get_max_abs_bias, but now it only finds the discriminated clusters
    min_new_size = get_min_cluster_size(full_data)
    
    if (max_abs_bias <= initial_bias) & (min_new_size > minimal_acceptable_cluster_size): #abs: >
        # add new cluster
        n_cluster = max(full_data['clusters'])
        full_data['clusters'][full_data['new_clusters'] == 1] =  n_cluster + 1
        
        x = get_next_cluster(full_data, metric)
        initial_bias = max_abs_bias
    else:
        x = get_random_cluster(full_data['clusters'])
        
print('done')

average_accuracy is:  0.7366666666666667
done


### 3. Analysing clustering results
Identifying cluster with most negative bias.

In [8]:
c = get_max_bias_cluster(full_data, metric)
max_bias = round(bias_acc(full_data, metric, c, "clusters"), 2)
highest_biased_cluster = full_data[full_data['clusters']==c]
print(f"cluster {c} has the highest negative bias: " + str(max_bias))
print("#elements in highest biased cluster:", len(highest_biased_cluster))

0 has bias 0.06048906048906044
2 has bias -0.04020530367835751
1 has bias -0.07954545454545459
3 has bias 0.06785714285714295
cluster 1 has the highest negative bias: -0.08
#elements in highest biased cluster: 36


Accuracy of classifier on full dataset and on cluster with highest discrimination bias.

In [10]:
accuracy_full_data = bias(full_data, metric)
most_biased_cluster_kmeans_aware = bias_acc(full_data, metric, c, 'clusters')
full_data[full_data['clusters']==c]

print('General accuracy of classifier on this dataset:', accuracy_full_data)

General accuracy of classifier on this dataset: 0.7366666666666667


Select and print discriminated cluser.

In [16]:
c=c
discriminated_cluster = full_data[full_data['clusters']==c]
print('Number of instances in discriminated cluster:', len(discriminated_cluster))
print('Number of errors in discriminated clusters: ', len(discriminated_cluster.loc[discriminated_cluster['errors']==1]))
discriminated_cluster.head()

Number of instances in discriminated cluster: 36
Number of errors in discriminated clusters:  12


,month,credit_amount,investment_as_income_percentage,residence_since,age,number_of_credits,people_liable_for,sex,status=A11,status=A12,...,skill_level=A174,telephone=A191,telephone=A192,foreign_worker=A201,foreign_worker=A202,predicted_class,true_class,errors,clusters,new_clusters
12,2.350125,0.325527,0.877513,1.084768,-0.137714,-0.711644,2.236068,-0.755434,-0.623610,1.687055,...,-0.452570,0.839372,-0.839372,0.195096,-0.195096,0,1.0,1.0,1,1
14,1.332510,0.099437,-0.976388,1.084768,0.301799,-0.711644,2.236068,-0.755434,1.603567,-0.592749,...,2.209605,-1.191367,1.191367,0.195096,-0.195096,1,0.0,1.0,1,1
17,2.350125,0.358585,0.877513,1.084768,0.917116,-0.711644,2.236068,-0.755434,1.603567,-0.592749,...,-0.452570,0.839372,-0.839372,0.195096,-0.195096,1,1.0,0.0,1,1
19,3.367741,4.481766,-0.049437,1.084768,2.147751,0.943341,-0.447214,1.323742,-0.623610,1.687055,...,2.209605,-1.191367,1.191367,0.195096,-0.195096,1,1.0,0.0,1,2
30,-0.193912,3.795519,-0.049437,1.084768,0.213896,-0.711644,-0.447214,1.323742,-0.623610,1.687055,...,2.209605,-1.191367,1.191367,0.195096,-0.195096,1,1.0,0.0,1,1


### 4. Statistical testing of inter-cluster difference 
Compute difference between cluster with highest discrimination bias and rest of dataset.

In [17]:
not_discriminated = full_data[full_data['clusters']!=c]
difference = (discriminated_cluster.mean()) - (not_discriminated.mean()) 
diff_dict = difference.to_dict()
difference

month                              0.732589
credit_amount                      0.843070
investment_as_income_percentage    0.353459
residence_since                    0.999226
age                                0.745284
                                     ...   
predicted_class                    0.082071
true_class                         0.232323
errors                             0.079545
clusters                          -0.234848
new_clusters                       2.000000
Length: 63, dtype: float64

Unscaling the data. Print summary of cluster with highest discrimination bias and the rest of data.

In [18]:
# unscaling the discriminated cluster
unscaled_discriminated = full_data.loc[discriminated_cluster.index, :]

# unscaled other data
unscaled_remaining = full_data.drop(discriminated_cluster.index)

display(unscaled_remaining.describe())
display(unscaled_discriminated.describe())

print(discriminated_cluster.index)

,month,credit_amount,investment_as_income_percentage,residence_since,age,number_of_credits,people_liable_for,sex,status=A11,status=A12,...,skill_level=A174,telephone=A191,telephone=A192,foreign_worker=A201,foreign_worker=A202,predicted_class,true_class,errors,clusters,new_clusters
count,264.000000,264.000000,264.000000,264.000000,264.000000,264.000000,264.000000,264.000000,264.000000,264.000000,...,264.000000,264.000000,264.000000,264.000000,264.000000,264.000000,264.000000,264.000000,264.000000,264.0
mean,-0.087911,-0.101168,-0.042415,-0.119907,-0.089434,-0.009529,-0.050820,0.040008,-0.007761,-0.005527,...,-0.039126,0.000923,-0.000923,-0.026604,0.026604,0.223485,0.295455,0.253788,1.234848,-1.0
std,0.909520,0.862589,1.025644,0.990299,0.948899,1.002411,0.953916,1.012432,0.998052,0.998850,...,0.966053,1.001736,1.001736,1.065253,1.065253,0.417372,0.457113,0.436004,1.084701,0.0
min,-1.381130,-1.006689,-1.903339,-1.688786,-1.368349,-0.711644,-0.447214,-0.755434,-0.623610,-0.592749,...,-0.452570,-1.191367,-0.839372,-5.125693,-0.195096,0.000000,0.000000,0.000000,0.000000,-1.0
25%,-0.702720,-0.649791,-0.976388,-0.764268,-0.840934,-0.711644,-0.447214,-0.755434,-0.623610,-0.592749,...,-0.452570,-1.191367,-0.839372,0.195096,-0.195096,0.000000,0.000000,0.000000,0.000000,-1.0
50%,-0.193912,-0.395679,-0.049437,0.160250,-0.313519,-0.711644,-0.447214,-0.755434,-0.623610,-0.592749,...,-0.452570,0.839372,-0.839372,0.195096,-0.195096,0.000000,0.000000,0.000000,2.000000,-1.0
75%,0.314895,0.157860,0.877513,1.084768,0.477604,0.943341,-0.447214,1.323742,1.603567,1.687055,...,-0.452570,0.839372,1.191367,0.195096,-0.195096,0.000000,1.000000,1.000000,2.000000,-1.0
max,2.350125,3.423897,0.877513,1.084768,3.466288,4.253312,2.236068,1.323742,1.603567,1.687055,...,2.209605,0.839372,1.191367,0.195096,5.125693,1.000000,1.000000,1.000000,3.000000,-1.0


,month,credit_amount,investment_as_income_percentage,residence_since,age,number_of_credits,people_liable_for,sex,status=A11,status=A12,...,skill_level=A174,telephone=A191,telephone=A192,foreign_worker=A201,foreign_worker=A202,predicted_class,true_class,errors,clusters,new_clusters
count,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,...,36.000000,36.000000,36.000000,36.000000,3.600000e+01,36.000000,36.000000,36.000000,36.0,36.000000
mean,0.644678,0.741902,0.311043,0.879319,0.655850,0.069877,0.372678,-0.293395,0.056917,0.040530,...,0.286923,-0.006769,0.006769,0.195096,-1.950956e-01,0.305556,0.527778,0.333333,1.0,1.000000
std,1.367346,1.528728,0.744131,0.546332,1.141377,1.007551,1.253566,0.876658,1.040485,1.035616,...,1.209311,1.015369,1.015369,0.000000,2.814929e-17,0.467177,0.506309,0.478091,0.0,0.338062
min,-1.211527,-0.869136,-0.976388,-0.764268,-1.104641,-0.711644,-0.447214,-0.755434,-0.623610,-0.592749,...,-0.452570,-1.191367,-0.839372,0.195096,-1.950956e-01,0.000000,0.000000,0.000000,1.0,0.000000
25%,-0.511917,-0.547006,-0.049437,1.084768,-0.181665,-0.711644,-0.447214,-0.755434,-0.623610,-0.592749,...,-0.452570,-1.191367,-0.839372,0.195096,-1.950956e-01,0.000000,0.000000,0.000000,1.0,1.000000
50%,0.314895,0.342056,0.877513,1.084768,0.521555,-0.711644,-0.447214,-0.755434,-0.623610,-0.592749,...,-0.452570,0.839372,-0.839372,0.195096,-1.950956e-01,0.000000,1.000000,0.000000,1.0,1.000000
75%,1.968520,1.434028,0.877513,1.084768,1.642311,0.943341,2.236068,-0.755434,1.603567,1.687055,...,2.209605,0.839372,1.191367,0.195096,-1.950956e-01,1.000000,1.000000,1.000000,1.0,1.000000
max,3.367741,4.481766,0.877513,1.084768,3.026776,2.598327,2.236068,1.323742,1.603567,1.687055,...,2.209605,0.839372,1.191367,0.195096,-1.950956e-01,1.000000,1.000000,1.000000,1.0,2.000000


Index([ 12,  14,  17,  19,  30,  36,  40,  43,  54,  62,  80,  85,  98, 112,
       116, 126, 136, 148, 149, 152, 156, 170, 181, 182, 192, 199, 223, 232,
       241, 245, 250, 257, 272, 278, 284, 297],
      dtype='int64')


In [19]:
unscaled_discriminated.head()

,month,credit_amount,investment_as_income_percentage,residence_since,age,number_of_credits,people_liable_for,sex,status=A11,status=A12,...,skill_level=A174,telephone=A191,telephone=A192,foreign_worker=A201,foreign_worker=A202,predicted_class,true_class,errors,clusters,new_clusters
12,2.350125,0.325527,0.877513,1.084768,-0.137714,-0.711644,2.236068,-0.755434,-0.623610,1.687055,...,-0.452570,0.839372,-0.839372,0.195096,-0.195096,0,1.0,1.0,1,1
14,1.332510,0.099437,-0.976388,1.084768,0.301799,-0.711644,2.236068,-0.755434,1.603567,-0.592749,...,2.209605,-1.191367,1.191367,0.195096,-0.195096,1,0.0,1.0,1,1
17,2.350125,0.358585,0.877513,1.084768,0.917116,-0.711644,2.236068,-0.755434,1.603567,-0.592749,...,-0.452570,0.839372,-0.839372,0.195096,-0.195096,1,1.0,0.0,1,1
19,3.367741,4.481766,-0.049437,1.084768,2.147751,0.943341,-0.447214,1.323742,-0.623610,1.687055,...,2.209605,-1.191367,1.191367,0.195096,-0.195096,1,1.0,0.0,1,2
30,-0.193912,3.795519,-0.049437,1.084768,0.213896,-0.711644,-0.447214,1.323742,-0.623610,1.687055,...,2.209605,-1.191367,1.191367,0.195096,-0.195096,1,1.0,0.0,1,1


####  Test to check statistical significance of inter-cluster difference (per feature)
Applying a Welch’s two-samples t-test for unequal variances to examine whether the differences in means for each feature are statistically significant and store results in a dictionary.

In [20]:
welch_dict = {}

features = [col for col in full_data.columns.tolist() if col not in ['scaled_errors','predicted_class','true_class','errors','clusters','new_clusters']]
    
for i in features:
    welch_i = stats.ttest_ind(unscaled_discriminated[i], unscaled_remaining[i], equal_var=False)
    
    # attach to dictionary
    welch_dict[i] = welch_i.pvalue
    
welch_dict

{'month': 0.003363431939954419,
 'credit_amount': 0.0024869792776754597,
 'investment_as_income_percentage': 0.013943757590349522,
 'residence_since': 1.3331851158881086e-13,
 'age': 0.0005442595243740106,
 'number_of_credits': 0.6592739970936896,
 'people_liable_for': 0.05791029573574221,
 'sex': 0.04103038856197385,
 'status=A11': 0.7268330984331791,
 'status=A12': 0.8026903048629718,
 'status=A13': 0.8190025136696473,
 'status=A14': 0.4804363529567879,
 'credit_history=A30': 0.2750750730667081,
 'credit_history=A31': 0.09847937056618611,
 'credit_history=A32': 0.006675437084000806,
 'credit_history=A33': 0.363823204159065,
 'credit_history=A34': 0.14881121558276791,
 'purpose=A40': 0.44265434365586476,
 'purpose=A41': 0.013547923882232357,
 'purpose=A410': 0.19173938220284065,
 'purpose=A42': 0.0011616213486439883,
 'purpose=A43': 2.1278468147922924e-07,
 'purpose=A44': 0.025066333426242163,
 'purpose=A45': 0.18306892525856797,
 'purpose=A46': 0.05170743578291297,
 'purpose=A48': 0.

#### p-values
A small p-value (p < 0.05) indicates that it is unlikely to observe inter-cluster difference due to chance. Sort difference on statistical significance (p-value).

In [23]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option("display.max_rows", None, "display.max_columns", None)
cluster_analysis_df = pd.DataFrame([diff_dict, welch_dict]).T
cluster_analysis_df.columns = ['difference','p-value']
cluster_analysis_df = cluster_analysis_df.sort_values('p-value')
cluster_analysis_df.head(10)

,difference,p-value
property=A124,2.81645,0.00000
housing=A152,-1.77806,0.00000
property=A123,-0.79751,0.00000
property=A121,-0.74983,0.00000
property=A122,-0.61522,0.00000
housing=A153,2.87956,0.00000
employment=A72,-0.50820,0.00000
residence_since,0.99923,0.00000
purpose=A43,-0.59982,0.00000
other_debtors=A101,0.33510,0.00000


#### Conclusion
Suspected negative bias by the classifier on the basis of (p<0.05):
- Housing (`housing=A151,A152,A153`);
- Property real estate or unknown (`property=A121,A124`);
- Other debtors (`other_debtors=A101,A102,A103`);
- Employment (`employment=A71`);
- Installment plans (`installment_plans=A142,A=143`);
- Foreign worker (`foreign_worker=A201,A202`);
- Month (`month`);
- Investment as percentage of annual income (`investment_as_income_percentage`);
- Gender (`sex`);
- Present residence since (`residence since`);
- Age in years (`age`)
- Type of property (`property=A42,A44,A45`);
- Job type (`skill_level=A174`);
- Amount of credit requested (`credit_amount`).

This means that loan applicants with the above features on average are significantly less often approved by the XGBoost classifier. An exact description of the features can be found [here](https://archive.ics.uci.edu/ml/datasets/Statlog+%28German+Credit+Data%29).

#### What's next?
Qualitative assessment with the help of subject matter experts to verify the measured quantitaive disparities. Additionally, sensitivity testing would be beneficial to shed light into the robustness of the bias scan tool. 